#### This notebook was made to create (and submit jobs for) JankyPipe, a pipeline that takes fastq files as input of Mycobacterium tuberculosis isolates, aligns the reads to H37Rv and calls variants. The output is a VCF file, a lineage call and a Qualimap report. This notebook also submits a job that runs JankyPipe on all of the *Longitudinal* isolates in our study.

In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
%matplotlib inline

In [2]:
import os
import pandas as pd
import numpy as np
from slurmpy import Slurm
import vcf
import shutil

### *Function* to launch JankyPipe as a Job

In [3]:
def Launch_JankyPipe(fqf1 , fqf2 , tag , output_dir , scratch_dir , O2_SLURM_logs_dir):
    
    '''
    This script launches a job to call variants for the input fastq files against H37Rv
    using a number of packages. The important output (VCF, lineage info files, quality report)
    is stored in the output directory while the intermediary files (SAMs, trimmed fastqs, BAM, etc)
    are stored in a scratch directory.
    '''
    
    #store all commands in a list
    commands_list = []
    
    #change directory to scratch
    commands_list.append( 'cd ' + scratch_dir )

    ###################################
    ### Load Necessary Modules ########
    ###################################

    #load perl
    commands_list.append( 'module load perl/5.24.0' )

    #load java
    commands_list.append( 'module load java/jdk-1.8u112' )

    #load BWA
    commands_list.append( 'module load bwa/0.7.15' )

    #load Samtools
    commands_list.append( 'module load samtools/1.3.1' )

    #load BCFtools
    commands_list.append( 'module load bcftools/1.3.1' )

    #load Picard
    commands_list.append( 'module load picard/2.8.0' )

    #Create Index files for Reference Genome
    commands_list.append( 'mkdir RefGen' )

    #copy reference genome over to RefGen folder
    commands_list.append( 'cp /home/rv76/Farhat_Lab/Reference_Seqs/H37Rv/h37rv.fasta RefGen/TBRefGen.fasta' )

    #change directory to RefGen folder
    commands_list.append( 'cd RefGen' )

    ###################################
    ### Create Index Files for H37Rv ##
    ###################################
    commands_list.append( 'samtools faidx TBRefGen.fasta' )
    commands_list.append( 'bwa index TBRefGen.fasta' )

    RefGen = scratch_dir + '/RefGen/TBRefGen.fasta' #H37Rv reference

    #go back to parent directory
    commands_list.append( 'cd ..' )

    ###################################
    ### UnZip FastQ files #############
    ###################################
    fqf1_base_name = fqf1.split('/')[-1][0:-9]
    fqf2_base_name = fqf2.split('/')[-1][0:-9]

    #work with the unzipped files for the rest of the pipeline (after unzipping them)
    fqf1_unzipped = scratch_dir + '/{}'.format(fqf1_base_name) + '.fastq'
    fqf2_unzipped = scratch_dir + '/{}'.format(fqf2_base_name) + '.fastq'

    commands_list.append( 'zcat {0} > {1}'.format(fqf1, fqf1_unzipped) )
    commands_list.append( 'zcat {0} > {1}'.format(fqf2, fqf2_unzipped) )

    #use the unzipped fastq files now
    fqf1 = fqf1_unzipped
    fqf2 = fqf2_unzipped

    ####################################
    ### PRINSEQ (trim reads) ##########
    ###################################

    #create directory for prinseq in output directory
    commands_list.append( 'mkdir ' + output_dir + '/prinseq' )

    commands_list.append( 'perl /n/data1/hms/dbmi/farhat/bin/prinseq-lite-0.20.4/prinseq-lite.pl -fastq {0} -fastq2 {1} -out_format 3 -out_good {2}/{3}-trimmed -out_bad null -log {4}/{3}-trimmed.log -min_qual_mean 20 -verbose'.format(fqf1, fqf2, scratch_dir, tag , output_dir+'/prinseq') )

    #use newly trimmed fastq files now
    fqf1 = scratch_dir + '/{}'.format(tag) + '-trimmed_1.fastq'
    fqf2 = scratch_dir + '/{}'.format(tag) + '-trimmed_2.fastq'

    ######################################
    ### BWA (align reads to reference) ###
    ######################################

    #create SAM file
    samfile = scratch_dir + '/{}.sam'.format(tag)

    #run BWA
    commands_list.append( 'bwa mem -M {3} {0} {1} > {2}'.format(fqf1 , fqf2 , samfile , RefGen) )

    #####################################
    ### PICARD (sort & convert to BAM) ##
    #####################################

    #create BAM file
    bamfile = scratch_dir + '/{0}.sorted.bam'.format(tag)

    commands_list.append( 'java -Xmx16G -jar /n/data1/hms/dbmi/farhat/bin/picard/picard/build/libs/picard.jar SortSam INPUT={0} OUTPUT={1} SORT_ORDER=coordinate'.format(samfile, bamfile) )

    ####################################
    ### PICARD (remove duplicates) ####
    ###################################

    #create BAM file with removed duplicates
    drbamfile = bamfile.replace(".bam", ".duprem.bam")

    #remove duplicates from BAM file
    commands_list.append( "java -Xmx32G -jar /n/data1/hms/dbmi/farhat/bin/picard/picard/build/libs/picard.jar MarkDuplicates I={0} O={1} REMOVE_DUPLICATES=true M={2} ASSUME_SORT_ORDER=coordinate".format(bamfile, drbamfile, drbamfile[:-4]+'.metrics') )

    ####################################
    ### SAMTOOLS (to index BAM file) ###
    ####################################
    
    commands_list.append( "samtools index {0}".format(drbamfile) )
    
    ######################################
    ### QUALIMAP (quality of BAM file) ###
    ######################################
    
    #store quality report, pilon VCF & lineage call information all in Output directory
    commands_list.append( 'cd ' + output_dir )
    commands_list.append( 'mkdir QualiMap' ) #make a folder for pilon output in output directory
    commands_list.append( 'unset DISPLAY' ) #unset JAVA virtual machine variable [http://qualimap.bioinfo.cipf.es/doc_html/faq.html]
    commands_list.append( "/n/data1/hms/dbmi/farhat/bin/qualimap_v2.2.1/qualimap bamqc -bam {0} --outdir {1} --outfile {2}.pdf --outformat PDF".format(drbamfile, output_dir+'/QualiMap', tag+'_stats') )

    ###################################
    ### PILON (call variants) #########
    ###################################
    
    #store quality report, pilon VCF & lineage call information all in Output directory
    commands_list.append( 'mkdir pilon' ) #make a folder for pilon output in output directory
    out_pilon_dir = output_dir + '/pilon/' #variable for pilon output path

    commands_list.append( 'java -Xmx32G -jar /n/data1/hms/dbmi/farhat/bin/pilon/pilon-1.22.jar --genome {0} --bam {1} --output {2} --outdir {3} --variant'.format(RefGen, drbamfile, tag, out_pilon_dir) )

    #####################################
    ### Luca's LINEAGE CALLING script ###
    #####################################

    #create directory 
    commands_list.append( 'mkdir ' + scratch_dir + '/fast-lineage-caller/' )#make a folder for lineage call in output directory
    commands_list.append( 'mkdir ' + output_dir + '/fast-lineage-caller/' )#make a folder for lineage call in scratch directory

    #create VRT file
    vrtfile = scratch_dir + '/fast-lineage-caller/{}.vrt'.format(tag)

    commands_list.append( 'cd ' + scratch_dir + '/fast-lineage-caller' )#change directory to store output in scratch

    #convert VCF to VRT
    commands_list.append( 'vrtTools-vcf2vrt.py {0} {1} 1'.format(out_pilon_dir+tag+'.vcf', vrtfile) )

    #call lineage with SNP database an VRT file
    commands_list.append( 'cd ' + output_dir + '/fast-lineage-caller' )#change directory to store output in VCF output

    commands_list.append( 'FastLineageCaller-assign2lineage.py /home/rv76/Bio_Pipelines/fast-lineage-caller-master/example/db_snps.tsv ' + vrtfile + ' &> ' + 'lineage_call.txt' )

    ###############################################################################################################
    ######################################## SUBMIT as a job to O2 ################################################
    ###############################################################################################################
    
    #append all commands in a single string to be submitted as a job
    JankyPipe_job = ''
    for command_i in commands_list:
        JankyPipe_job = JankyPipe_job + '\n' + command_i
    
    #directory where you want output + error files
    os.chdir(O2_SLURM_logs_dir)

    job_name = tag

    s = Slurm(job_name , {'partition':'short' , 'n':'1' , 't':'0-6:00:00' , 'mem-per-cpu':'36G' , 'mail-type':'FAIL' , 'mail-user':'roger_vargas@g.harvard.edu'})

    #submits the job
    job_id = s.run(JankyPipe_job)

    print job_name  + ' : ' +  str(job_id)

### Longitudinal Samples

Pull all relevant sequenced isolate and corresponding FastQ file paths

In [4]:
sample_annotation = pd.read_csv('/n/data1/hms/dbmi/farhat/Roger/inhost_TB_dynamics_project/CSV_files/sample_annotation_files/cetr_casali_walker_trauner_witney_xu_guerra_bryant_fastq_path_names.csv' , sep = ',').set_index('patient_id')

In [5]:
sample_annotation.head(n=2)

,fastq_files,population,run_ID,sample_ID,sample_order
patient_id,,,,,
I0005973-8,/n/data1/hms/dbmi/farhat/cetr_strains/good_wgs...,CETR,NaN,Peru3062,1
I0005973-8,/n/data1/hms/dbmi/farhat/cetr_strains/good_wgs...,CETR,NaN,Peru3315,2


In [6]:
np.shape(sample_annotation)

(614, 5)

### Create directories for each isolate and launch JankyPipe

IMPORTANT PARENT DIRECTORIES 

-   /n/scratch2/rv76/inhost_TB_dynamics_project/JankyPipe/intermediary_files/

    [to store intermediate files (unzipped fastq, trimmed fastq, SAM, sorted BAM, etc)]


-   /n/data1/hms/dbmi/farhat/Roger/inhost_TB_dynamics_project/JankyPipe/output/

    [to store final files (pilon VCF, lineage, QualiMap, trim logs)]


-   /n/data1/hms/dbmi/farhat/Roger/inhost_TB_dynamics_project/JankyPipe/O2_SLURM_logs/

    [to store submitted SLURM script, and SLURM error & verbose logs]

In [ ]:
for isolate_i in range(0 , np.shape(sample_annotation)[0]):
    
    isolate_fastq_paths = sample_annotation.iloc[isolate_i , 0]

    #paths & names for fastq files
    fqf1 = isolate_fastq_paths.split(';')[0]
    fqf2 = isolate_fastq_paths.split(';')[1]

    #get the tag ID for the fastq files (same as ID for fastq files)
    tag = fqf1.split('/')[-1].split('_')[0]

    #where pilon VCF and lineage information will be stored [LAB FOLDER]
    output_dir = '/n/data1/hms/dbmi/farhat/Roger/inhost_TB_dynamics_project/JankyPipe/output/' + tag
    if os.path.exists(output_dir):
        shutil.rmtree(output_dir)
        os.makedirs(output_dir)
    elif not os.path.exists(output_dir):
        os.makedirs(output_dir)
        
    #where everything else happens (trimming, aligning, etc.) [SCRATCH FOLDER]
    scratch_dir = '/n/scratch2/rv76/inhost_TB_dynamics_project/JankyPipe/intermediary_files/' + tag
    if os.path.exists(scratch_dir):
        shutil.rmtree(scratch_dir)
        os.makedirs(scratch_dir)
    elif not os.path.exists(scratch_dir):
        os.makedirs(scratch_dir)

    #store O2 job log files [LAB FOLDER]
    O2_SLURM_logs_dir = '/n/data1/hms/dbmi/farhat/Roger/inhost_TB_dynamics_project/JankyPipe/O2_SLURM_logs/' + tag

    if os.path.exists(O2_SLURM_logs_dir):
        shutil.rmtree(O2_SLURM_logs_dir)
        os.makedirs(O2_SLURM_logs_dir)
    elif not os.path.exists(O2_SLURM_logs_dir):
        os.makedirs(O2_SLURM_logs_dir)

    #Launch JankyPipe after making necessary directories!!!
    Launch_JankyPipe(fqf1 , fqf2 , tag , output_dir , scratch_dir , O2_SLURM_logs_dir)

#### save tags (corresponds to folder names)

In [7]:
#store tags to each sample
tag_list = []

for isolate_i in range(0 , np.shape(sample_annotation)[0]):
    
    isolate_fastq_paths = sample_annotation.iloc[isolate_i , 0]

    #paths & names for fastq files
    fqf1 = isolate_fastq_paths.split(';')[0]

    #get the tag ID for the fastq files (same as ID for fastq files)
    tag = fqf1.split('/')[-1].split('_')[0]

    tag_list.append(tag)
    
sample_annotation['tag'] = tag_list 

#store as CSV
sample_annotation.to_csv('/n/data1/hms/dbmi/farhat/Roger/inhost_TB_dynamics_project/CSV_files/sample_annotation_files/cetr_casali_walker_trauner_witney_xu_guerra_bryant_fastq_path_names_and_JankyPipe_tags.csv' , sep = ',')

In [8]:
sample_annotation.head(n=2)

,fastq_files,population,run_ID,sample_ID,sample_order,tag
patient_id,,,,,,
I0005973-8,/n/data1/hms/dbmi/farhat/cetr_strains/good_wgs...,CETR,NaN,Peru3062,1,Peru3062
I0005973-8,/n/data1/hms/dbmi/farhat/cetr_strains/good_wgs...,CETR,NaN,Peru3315,2,Peru3315


### Determine if jobs ran successfully or not

In [9]:
successful_run = []

for isolate_i in range(0 , np.shape(sample_annotation)[0]):

    #get the tag ID for the fastq files (same as ID for fastq files)
    tag = sample_annotation.tag[isolate_i]

    #where pilon VCF and lineage information will be stored [LAB FOLDER]
    output_dir = '/n/data1/hms/dbmi/farhat/Roger/inhost_TB_dynamics_project/JankyPipe/output/' + tag
    
    #check to see 'Lineage Call' folder exists in the output directory (last thing that is run in JankyPipe)
    if os.path.exists(output_dir + '/fast-lineage-caller/'):
        
        successful_run.append('yes')
        
    else:
        
        successful_run.append('no')
        
sample_annotation['successful_run'] = successful_run

In [10]:
sample_annotation[sample_annotation.successful_run == 'no']

,fastq_files,population,run_ID,sample_ID,sample_order,tag,successful_run
patient_id,,,,,,,
KPS_1,/n/data1/hms/dbmi/farhat/fastq_db/guerra/ERR18...,GUERRA,ERR182021,NaN,1,ERR182021,no
KPS_1,/n/data1/hms/dbmi/farhat/fastq_db/guerra/ERR18...,GUERRA,ERR181711,NaN,2,ERR181711,no
KPS_2,/n/data1/hms/dbmi/farhat/fastq_db/guerra/ERR21...,GUERRA,ERR212067,NaN,1,ERR212067,no
KPS_2,/n/data1/hms/dbmi/farhat/fastq_db/guerra/ERR77...,GUERRA,ERR773785,NaN,2,ERR773785,no
KPS_3,/n/data1/hms/dbmi/farhat/fastq_db/guerra/ERR21...,GUERRA,ERR212061,NaN,1,ERR212061,no
KPS_3,/n/data1/hms/dbmi/farhat/fastq_db/guerra/ERR21...,GUERRA,ERR212068,NaN,2,ERR212068,no
KPS_4,/n/data1/hms/dbmi/farhat/fastq_db/guerra/ERR19...,GUERRA,ERR190350,NaN,1,ERR190350,no
KPS_4,/n/data1/hms/dbmi/farhat/fastq_db/guerra/ERR16...,GUERRA,ERR163987,NaN,2,ERR163987,no
KPS_5,/n/data1/hms/dbmi/farhat/fastq_db/guerra/ERR03...,GUERRA,ERR037522,NaN,1,ERR037522,no


### Re-Run isolates through pipeline that hit a run-timelimit

In [11]:
#isolates that don't have a lineage-call directory didn't finish running through pipeline
sample_annotation_ReRun = sample_annotation[sample_annotation.successful_run == 'no']

#if path already exists, remove current contents, then recreate empty directory
#if path doesn't exist, create new directory

for isolate_i in range(0 , np.shape(sample_annotation_ReRun)[0]):
    
    isolate_fastq_paths = sample_annotation_ReRun.iloc[isolate_i , 0]

    #paths & names for fastq files
    fqf1 = isolate_fastq_paths.split(';')[0]
    fqf2 = isolate_fastq_paths.split(';')[1]

    #get the tag ID for the fastq files (same as ID for fastq files)
    tag = fqf1.split('/')[-1].split('_')[0]

    #where pilon VCF and lineage information will be stored [LAB FOLDER]
    output_dir = '/n/data1/hms/dbmi/farhat/Roger/inhost_TB_dynamics_project/JankyPipe/output/' + tag
    
    if os.path.exists(output_dir):
        shutil.rmtree(output_dir)
        os.makedirs(output_dir)
    elif not os.path.exists(output_dir):
        os.makedirs(output_dir)
        

    #where everything else happens (trimming, aligning, etc.) [SCRATCH FOLDER]
    scratch_dir = '/n/scratch2/rv76/inhost_TB_dynamics_project/JankyPipe/intermediary_files/' + tag
    
    if os.path.exists(scratch_dir):
        shutil.rmtree(scratch_dir)
        os.makedirs(scratch_dir)
    elif not os.path.exists(scratch_dir):
        os.makedirs(scratch_dir)

        
    #store O2 job log files [LAB FOLDER]
    O2_SLURM_logs_dir = '/n/data1/hms/dbmi/farhat/Roger/inhost_TB_dynamics_project/JankyPipe/O2_SLURM_logs/' + tag
    
    if os.path.exists(O2_SLURM_logs_dir):
        shutil.rmtree(O2_SLURM_logs_dir)
        os.makedirs(O2_SLURM_logs_dir)
    elif not os.path.exists(O2_SLURM_logs_dir):
        os.makedirs(O2_SLURM_logs_dir)

        
    #Launch JankyPipe after making necessary directories!!!
    Launch_JankyPipe(fqf1 , fqf2 , tag , output_dir , scratch_dir , O2_SLURM_logs_dir)

submitted: Submitted batch job 31784984
submitted: Submitted batch job 31784986
submitted: Submitted batch job 31784988
submitted: Submitted batch job 31784990


ERR182021 : 31784984
ERR181711 : 31784986
ERR212067 : 31784988
ERR773785 : 31784990


submitted: Submitted batch job 31784992
submitted: Submitted batch job 31784994
submitted: Submitted batch job 31784996
submitted: Submitted batch job 31784998


ERR212061 : 31784992
ERR212068 : 31784994
ERR190350 : 31784996
ERR163987 : 31784998


submitted: Submitted batch job 31785000


...

submitted: Submitted batch job 31785299
submitted: Submitted batch job 31785300


ERR108125 : 31785299
ERR171144 : 31785300


submitted: Submitted batch job 31785301
submitted: Submitted batch job 31785302
submitted: Submitted batch job 31785303
submitted: Submitted batch job 31785304


ERR171145 : 31785301
ERR171154 : 31785302
ERR171155 : 31785303
ERR171156 : 31785304
ERR171157 : 31785305
ERR171158 : 31785306
ERR171159 : 31785307


submitted: Submitted batch job 31785305
submitted: Submitted batch job 31785306
submitted: Submitted batch job 31785307


## Scrape and Analyze Mean Coverage

#### Import Sample Annotation file for filtered *longitudinal* isolates pairs

In [6]:
sample_annotation = pd.read_csv('/n/data1/hms/dbmi/farhat/Roger/inhost_TB_dynamics_project/CSV_files/sample_annotation_files/Longitudinal_fastq_path_names_and_JankyPipe_tags_filtered_final.csv' , sep  = ',').set_index('patient_id')

In [7]:
sample_annotation.head()

,fastq_files,population,run_ID,sample_ID,sample_order,tag,isolate_type
patient_id,,,,,,,
P000183,/n/data1/hms/dbmi/farhat/fastq_db/walker/ERR03...,WALKER,ERR039337,NaN,1,ERR039337,longitudinal
P000183,/n/data1/hms/dbmi/farhat/fastq_db/walker/ERR03...,WALKER,ERR039338,NaN,2,ERR039338,longitudinal
1960,/n/data1/hms/dbmi/farhat/cetr_strains/good_wgs...,CETR,MMJA00000000,Peru5115,1,Peru5115,longitudinal
1960,/n/data1/hms/dbmi/farhat/cetr_strains/good_wgs...,CETR,MMPC00000000,Peru4668,2,Peru4668,longitudinal
2491,/n/data1/hms/dbmi/farhat/cetr_strains/good_wgs...,CETR,MMZT00000000,Peru4903,1,Peru4903,longitudinal


In [8]:
np.shape(sample_annotation)

(400, 7)

In [14]:
from itertools import compress
import time
import sys
import pickle
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.ticker as ticker
from pylab import plot, show, savefig, xlim, figure, hold, ylim, legend, boxplot, setp, axes
from itertools import compress
from pylab import MaxNLocator
import seaborn as sns; sns.set()
from matplotlib.colors import LogNorm
from matplotlib import gridspec
import ast
import itertools
import seaborn as sns
from sklearn.preprocessing import StandardScaler

In [27]:
#genomic data directory
rolling_DB_dir = '/n/data1/hms/dbmi/farhat/Roger/inhost_TB_dynamics_project/JankyPipe/output/'

#get all folders (each folder corresponds to a different sequenced isolate)
isolate_directories = list(sample_annotation.tag)

#dictionary that stores the mean coverage for each isolate (that successfully ran through megapipe2.0) from QUALIMAP
mean_coverage_dict = {} #key: isolate_ID , value: mean coverage

#iterate through each sequenced isolate
isolate_i = 0

for isolate_ID in isolate_directories:

    #directory that stores files for each sequenced isolate
    directory_for_sequenced_isolate = rolling_DB_dir + isolate_ID

    #check to see if megapipe successfully ran on sequenced isolate
    try:
        #existence of a PILON and QUALIMAP directories and corresponding VCF file [there's also an option for FAST-LINEAGE-CALLER]
        if ( 'pilon' in os.listdir(directory_for_sequenced_isolate) ) and ( 'QualiMap' in os.listdir(directory_for_sequenced_isolate) ):
            
            #existence of a VCF and GENOME-QUALITY files in relevent directories [there's also an option for LINEAGE]
            if ( 'vcf' in list( itertools.chain.from_iterable( [filename.split('.') for filename in os.listdir(directory_for_sequenced_isolate + '/pilon/')] ) ) ) and ( 'genome_results.txt' in os.listdir(directory_for_sequenced_isolate + '/QualiMap/') ):
                
                #we have a valid VCF and Quality-Map (and Lineage file?) file so megapipe ran successfully, let's keep the variant call information for this sequenced isolate and look for qualimap, lineage call data as well
                
                #QUALIMAP DATA
                ########################################################################################################################
                #look for qualimap output txt file that has mean coverage & mean read length
                qualimap_BAM_file_stats = directory_for_sequenced_isolate + '/QualiMap/' + 'genome_results.txt'

                #parse qualimap txt file and store the mean coverage for the BWA mapping (BAM file) & mean read length
                with open(qualimap_BAM_file_stats ,'r') as f:

                    #iterate through lines in text file
                    for stat_per_line in f:

                        #find the mean coverage for mapping
                        if 'mean coverageData' in stat_per_line:
                            
                            mean_coverage = float( stat_per_line.split('=')[-1][:-2].replace(',' , '') )
                            mean_coverage_dict[isolate_ID] = mean_coverage
                            
                            break #once we have mean coverage
                ########################################################################################################################
                
                #keep track of progress   
                isolate_i += 1
                if isolate_i % np.ceil(0.05*len(isolate_directories)) == 0:
                    print float(isolate_i) / float(len(isolate_directories))
        
    except OSError: #hit some file that is not another directory with genomic data 
        continue

0.05
0.1
0.15
0.2
0.25
0.3
0.35
0.4
0.45
0.5
0.55
0.6
0.65
0.7
0.75
0.8
0.85
0.9
0.95


In [28]:
mean_coverage_DF = pd.DataFrame()

mean_coverage_series = pd.Series(mean_coverage_dict)

mean_coverage_DF['mean_coverage'] = mean_coverage_series
mean_coverage_DF['isolate_ID'] = mean_coverage_DF.index

In [29]:
mean_coverage_DF.head()

,mean_coverage,isolate_ID
ERR025846,144.2826,ERR025846
ERR025847,94.7828,ERR025847
ERR036189,281.9261,ERR036189
ERR036195,139.9493,ERR036195
ERR037467,244.1663,ERR037467


In [30]:
np.shape(mean_coverage_DF)

(397, 2)

Average Coverage across isolates

In [32]:
mean_coverage_DF.mean_coverage.mean()

185.900731234257